In [15]:
import os
import numpy as np
import time
import sys
from skimage.external import tifffile
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import clear_output
import pandas as pd
from ChexnetTrainer import ChexnetTrainer
import torch

working_dir = '/mnt/storage/mlhc_project'
def bdir(fname):
    return os.path.join(working_dir, fname)

In [3]:
# Need to generate the txt file for train and validation
txt_file_path = './dataset/train.txt'
labels_paths = ['../data_4/labels_20k_4.npy', 
           '../data_5/labels_5.npy', 
           '../data_6/labels_6.npy',
               '../data_2/labels_2.npy',
               '../data_7/labels_7.npy',
               '../data_8/labels_8.npy']
img_dirs = ['../data_4/data/tiffs_20k_4', 
           '../data_5/data/tiffs_5', 
           '../data_6/data/tiffs_6',
           '../data_2/data/tiffs_2',
           '../data_7/data/tiffs_7',
           '../data_8/data/tiffs_8']

########################

labels = []
for labels_path in labels_paths:
    labels.append(np.load(labels_path, encoding='bytes'))
total_labels = np.concatenate(tuple(labels), axis=0)


names_list = []
for img_dir in img_dirs:
    temp_names_list = [' ']*len(os.listdir(img_dir))
    temp_dir_header = img_dir[3:]
    print(temp_dir_header)
    for fname in os.listdir(img_dir):
        # Get actual index 
        index = int(fname.split('_')[0])
        temp_names_list[index] = os.path.join(temp_dir_header, fname)
    names_list += temp_names_list
    
d = {'filename':names_list}
df1 = pd.DataFrame(data=d)
df2 = pd.DataFrame(data=total_labels)
df2 = df2.astype('int')

df = pd.concat((df1,df2),axis=1)

df.to_csv('./dataset/train.txt',sep=' ', index = False, header=False)

data_4/data/tiffs_20k_4
data_5/data/tiffs_5
data_6/data/tiffs_6
data_2/data/tiffs_2
data_7/data/tiffs_7
data_8/data/tiffs_8


In [4]:
# Need to generate the txt file for train and validation
txt_file_path = './dataset/valtest.txt'
labels_path = '../data_3/labels_20k_3.npy'
img_dir = '../data_3/data/tiffs_3'

########################
labels = np.load(labels_path, encoding='bytes')
#np.save(labels_path, labels.astype('int'))
names_list = [' ']*len(labels)
for fname in os.listdir(img_dir):
    # Get actual index 
    index = int(fname.split('_')[0])
    names_list[index] = fname
    
d = {'filename':names_list}
df1 = pd.DataFrame(data=d)
df2 = pd.DataFrame(data=labels)
df2 = df2.astype('int')

df = pd.concat((df1,df2),axis=1)

df.to_csv(txt_file_path,sep=' ', index = False, header=False)

In [5]:
# Convert valtest into val and test
val_txt_file_path = './dataset/val.txt'
test_txt_file_path = './dataset/test.txt'
index_to_split = 8712

df = pd.read_csv(txt_file_path, sep=' ', header=None)
df_val = df[:index_to_split]
df_test = df[index_to_split:]
df_val.to_csv(val_txt_file_path, sep=' ', index=False, header=False)
df_test.to_csv(test_txt_file_path, sep=' ', index=False, header=False)

In [12]:
# Training code 
def runTrain():
    checkpoint = 'models/m-09052019-153716.pth.tar' # if None, then we start training over. otherwise start training an old model 
    DENSENET121 = 'DENSE-NET-121'
    DENSENET169 = 'DENSE-NET-169'
    DENSENET201 = 'DENSE-NET-201'
    
    timestampTime = time.strftime("%H%M%S")
    timestampDate = time.strftime("%d%m%Y")
    timestampLaunch = timestampDate + '-' + timestampTime
    
    #---- Path to the directory with images
#     pathDirDataTrain = [bdir('data/dataset_20k.npy'), bdir('data/labels_20k.npy')]
#     pathDirDataVal = [bdir('data_2/dataset_20k_2.npy'), bdir('data_2/labels_20k_2.npy')]
#     pathDirDataTrain = bdir('data_4/data/tiffs_20k_4')
    pathDirDataTrain = bdir('')
    pathDirDataVal = bdir('data_3/data/tiffs_3')
#     pathDirDataVal = None
    
    #---- Paths to the files with training, validation and testing sets.
    #---- Each file should contains pairs [path to image, output vector]
    #---- Example: images_011/00027736_001.png 0 0 0 0 0 0 0 0 0 0 0 0 0 0
    pathFileTrain = 'dataset/train.txt'
    pathFileVal = 'dataset/val.txt'
    #pathFileTest = './dataset/test_1.txt'
    
    #---- Neural network parameters: type of the network, is it pre-trained 
    #---- on imagenet, number of classes
    nnArchitecture = DENSENET121
    nnIsTrained = True
    nnClassCount = 14
    
    #---- Training settings: batch size, maximum number of epochs
    trBatchSize = 16
    trMaxEpoch = 50
    
    #---- Parameters related to image transforms: size of the down-scaled image, cropped image
    imgtransResize = 224
    imgtransCrop = 224
        
    pathModel = 'm-' + timestampLaunch + '.pth.tar'
    
    print ('Training NN architecture = ', nnArchitecture)
    ChexnetTrainer.train(pathDirDataTrain, pathDirDataVal, pathFileTrain, pathFileVal, nnArchitecture, nnIsTrained, nnClassCount,
                         trBatchSize, trMaxEpoch, imgtransResize, imgtransCrop, timestampLaunch, checkpoint)
    
#     print ('Testing the trained model')
#     ChexnetTrainer.test(pathDirData, pathFileTest, pathModel, nnArchitecture, nnClassCount, nnIsTrained, trBatchSize, imgtransResize, imgtransCrop, timestampLaunch)


In [ ]:
runTrain()

Training NN architecture =  DENSE-NET-121


6661it [11:05,  4.35it/s]
545it [00:13, 40.04it/s]


Epoch [1] [save] [10052019-030645] val_loss= 0.26089737025012666, train_loss= 0.2427527137126423


6661it [11:17, 11.64it/s]
545it [00:14, 38.26it/s]


Epoch [2] [save] [10052019-031823] val_loss= 0.2602634498832423, train_loss= 0.24285946641776712


...

6661it [11:24,  9.73it/s]
545it [00:14, 38.90it/s]


Epoch [20] [----] [10052019-064940] val_loss= 0.2610368210104627, train_loss= 0.2414881917612364


6661it [11:27,  9.68it/s]
545it [00:12, 42.13it/s]


Epoch [21] [----] [10052019-070129] val_loss= 0.26003154153260616, train_loss= 0.2414450574363697


2617it [04:26,  8.93it/s]

In [10]:
def runTest():
    
    # Test set 
    pathDirData = bdir('data_3/data/tiffs_3')
    pathFileTest = 'dataset/test.txt'
    nnArchitecture = 'DENSE-NET-121'
    nnIsTrained = True
    nnClassCount = 14
    trBatchSize = 16
    imgtransResize = 224
    imgtransCrop = 224
    
    # 'models/m-08052019-184741.pth.tar' is the 20k model 
    pathModel = 'models/m-08052019-184741.pth.tar'

    
    timestampLaunch = ''
    print("Test set:")
    ChexnetTrainer.test(pathDirData, pathFileTest, pathModel, nnArchitecture, nnClassCount, nnIsTrained, trBatchSize, imgtransResize, imgtransCrop, timestampLaunch)
    
    
    # Val set 
    pathDirData = bdir('data_3/data/tiffs_3')
    pathFileTest = 'dataset/val.txt'
    print("Val set:")
    ChexnetTrainer.test(pathDirData, pathFileTest, pathModel, nnArchitecture, nnClassCount, nnIsTrained, trBatchSize, imgtransResize, imgtransCrop, timestampLaunch)
    
    
    # Train set 
    pathDirData = bdir('')
    pathFileTest = 'dataset/train.txt'
    print("Train set:")
    ChexnetTrainer.test(pathDirData, pathFileTest, pathModel, nnArchitecture, nnClassCount, nnIsTrained, trBatchSize, imgtransResize, imgtransCrop, timestampLaunch)
    

In [13]:
runTest()

Test set:


/home/webster/anaconda3/envs/hubris/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


No Finding   0.8471829900693314
Enlarged Cardiomediastinum   0.6878402575717024
Cardiomegaly   0.7920743586275965
Airspace Opacity   0.7293064029579733
Lung Lesion   0.6971406003159558
Edema   0.8759862186593894
Consolidation   0.779920146475776
Pneumonia   0.73054510033367
Atelectasis   0.7977155500733691
Pneumothorax   0.7940490516848622
Pleural Effusion   0.901310436177698
Pleural Other   0.787115292286807
Fracture   0.6463964316967726
Support Devices   0.8714726586320809
AUROC mean  0.7812810643684784
Val set:


/home/webster/anaconda3/envs/hubris/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


No Finding   0.8515557042175443
Enlarged Cardiomediastinum   0.7136849900647442
Cardiomegaly   0.7954346129374082
Airspace Opacity   0.7367577400150691
Lung Lesion   0.69655310560055
Edema   0.8762558122494
Consolidation   0.78930960335876
Pneumonia   0.710487906491001
Atelectasis   0.7959914912016632
Pneumothorax   0.8229949115580325
Pleural Effusion   0.9029066522564707
Pleural Other   0.7837479655893977
Fracture   0.6047700203729822
Support Devices   0.8765291033236826
AUROC mean  0.7826776915364944
Train set:


/home/webster/anaconda3/envs/hubris/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


KeyboardInterrupt: 